# DEM(数値標高モデル)からの水系図作成

ここでは、数値標高モデルのデータから水系図の作成を行います。

以下の2つの方法を説明します。

- QGISを経由してGRASS GISのモジュールを利用する
- PythonからGRASS GISのモジュールを利用する

どちらも、[GRASS GIS](https://grass.osgeo.org/)の[r.watershed](https://grass.osgeo.org/grass78/manuals/r.watershed.html)というモジュールを利用して水系図を作成します。GRASS GISは独自のデータベース構造を持つため、はじめは仕組みを理解することが難しいかもしれませんが、ラスタデータやベクタデータを扱う多くのライブラリがあり、リモートセンシングやGISの解析に非常に有用なツールです。QGISを経由してGRASSのモジュールを利用することで、比較的簡易にそれらのツールを利用することができるようになります。また、Pythonを使う方法を利用すると、バッチ処理やほかのPython処理とスムーズに結合することができるようになります。

まずはQGISを使った方法を説明します。

事前に、[OSGeo4W](https://trac.osgeo.org/osgeo4w/)からネットワークインストーラーをダウンロードして、QGIS Desktop with GRASSが使えるようにしておいてください。インストールの設定は、エクスプレスインストールを選択し、すべてデフォルトの設定で大丈夫です。

データは笛吹川上流域のSRTM DEMデータを利用します。SRTMデータはWGS84の地理座標系で提供されています。事前にUTM座標系に変換したものを準備したので、[ここから](data/watershed/dem_fuefuki.tif)ダウンロードしてdataフォルダに保存してください。

## QGISを用いた方法
スタートメニューから、QGISを検索し、**QGIS Desktop with GRASS**を起動してください。**with GRASS表記がないものでは以下は動きません。**

![QGISの起動](images/ss_catchment_01_start_qgis.png)

ダウンロードした`dem_fuefuki.tif`をQGISにドラッグ＆ドロップしてデータを読み込みます。読み込まれたデータはLayersパネルに一覧が表示されます。Layersパネルが表示されていない場合、View → PanelsメニューからLayersをオンにしてください。また、同じメニューから、Processing Tool Boxのパネルも表示してください。

<video controls src="../_static/mv_watershed_load_dem.mp4">animation</video>

### GRASS GISモジュールの実行
Processing Toolboxから、GRASS GISやSAGA GISなどのほかのフリーのGISソフトが提供するツール群を利用することができます。Processing Toolboxの検索ボックスに"water"と入れて表示される"r.watershed"モジュールを起動してください。

![watershedモジュールの起動](images/ss_watershed_rwatershed.png)

モジュールに以下の設定を行い、Runを押して実行してください。記載のない項目はすべてデフォルトの値です。

- Elevation : dem_fuefuki [EPSG:32654]
- Minimum size of exterior watershed basin [optional] : 1000

上記のオプションで最小面積が$1000m^2$となるような集水域図が作成されます。
ログメッセージが表示後、モジュールの実行が完了します。モジュールウィンドウは閉じてください。

### 結果
Layersパネルに計算されたラスターデータが追加されます。

作成されたラスタについて、以下で説明します。図はDEMから作成した陰影図を重ね合わせています。

#### Unique label for each watershed basin
集水域を表します。各集水域に固有のIDが振られています。  
![集水域](images/watershed_basin.png)

#### Half-basins
河川の左右別の集水域を表します。下流から見て右の支流に偶数、左の支流にその1少ない奇数のIDが振られています。  
![半集水域](images/watershed_basin_half.png)


#### Number of cells that drain through each cell
各ピクセルより上流にあるピクセルの数を表します。ピクセルサイズと掛け合わせることで、各ピクセルにおける集水域の面積となります。入力DEMの範囲外から流入する地域については値が負になっています。

![ピクセル数](images/watershed_n_cell.png)

#### Stream segments
河川の位置を表します。各河川は集水域のIDに対応するID値を持ちます。GRASSの`r.to.vect`モジュールの入力として与えることで、ベクターデータ(shapefile)として出力できます。

![stream](images/watershed_stream_seg.png)

#### Drainage direction
各ピクセルでの斜面の方位を示しています。方位は以下の図に示す値で定義されています。([参考](https://idea.isnew.info/how-to-import-arcgis-flow-direction-into-grass-gis.html))

![流下方位](images/watershed_drainage_direction_map.png)
![方位定義](images/watershed_drainage_direction.png)

#### Stream power index a * tan(b)
各地点における水が流下する力を示します。各ピクセルにおける単位等高線あたりの集水面積を$\alpha$、各地点での傾斜角度を$\beta$としたとき、$\alpha \times tan(\beta)$と定義されるStream Power Index(SPI)と呼ばれる指標です。(Moore et al. 1991)

![SPI](images/watershed_stream_pi.png)

#### Topographic index ln(a / tan(b))
各地点における水が流下する力を示します。SPIの自然対数で定義されます。(Quinn et al. 1991)

![TCI](images/watershed_topo_indx.png)

#### Slope length and steepness (LS) factor for USLE
Revised Universal Soil Loss Equation (RUSLE)におけるLSの値(傾斜長および傾斜度)を示します。

![LS](images/watershed_slope_ls.png)

#### Slope steepness (S) factor for USLE
Revised Universal Soil Loss Equation (RUSLE)におけるSの値(傾斜度)を示します。

![S](images/watershed_slope_s.png)

#### 結果の保存
Layerウィンドウ内でレイヤの右側に![メモリアイコン](images/watershed_memory.png)表示があるレイヤはQGISの一時ファイルに保存されています。GRASSモジュールを実行する際に計算結果の保存先を指定しなかった場合、デフォルトで一時ファイルに保存されます。これらはQGISを終了した際に削除されてしまうので、QGISを閉じる前にGeoTiffファイルとして保存してください。保存方法は、保存したいレイヤを右クリック、Export -> Save as... で表示されるダイヤログにおいて、フォーマットと保存先フォルダを指定します。その他の設定項目はデフォルト値で大丈夫です。ラスタがファイルに保存され、QGISのレイヤにも追加されます。

以上でQGISを通じたGRASSモジュールによる集水域図を作成することができました。

## PythonからGRASSモジュールを利用する
GRASS GISは独自のデータベースを使って計算を行います。Geotiffやシェープファイルなどの一般的なファイルは、初めにGRASSデータベースへ登録する必要があります。

各種の計算は、GRASSデータベースに登録されたデータに対して行われ、計算結果もデータベースに登録されます。

計算の成果物となるラスタデータやベクタデータはこのGRASSデータベース上にあるため、GeotiffやShapefile等の一般的なファイルとしてとりだす作業も必要になります。

上記のQGISを用いたGRASSモジュールの利用では、これらの操作がバックグラウンドで自動で行われます。

GRASS GISはもともとBashターミナルからの操作を想定して設計されたプログラムですが、GRASS GIS 7.8以降ではPythonからの利用が非常にしやすくなりました。

以降では、GRASSが提供するPythonインターフェースを用いて、Jupyter Notebook上でGRASS GISを利用してみます。
[参考リンク](https://grasswiki.osgeo.org/wiki/Working_with_GRASS_without_starting_it_explicitly)

Windows 10のパソコンから利用する場合、[Jupyter Notebookと環境構築](setup_ipynb.ipynb)を参考にPython環境をセットアップしてください。残念ながら、Google CodelaboratoryではGRASSのバージョンが古いため、以降のコマンドは動きません。

Python環境が設定出来たら、ターミナルから以下のコマンドを実行し、GRASS GISをインストールします。15分ほどかかります。
```bash
# Ubuntuのパスワード入力が必要
sudo apt install grass
```
GRASSのバージョンは7.8以上であることを以下のコマンドで確認してください。
```bash
grass --version
```
Jupyter Notebookからは以下のように確認することもできます。

In [1]:
!grass --version

GRASS GIS 7.8.2

Geographic Resources Analysis Support System (GRASS) is Copyright,
1999-2019 by the GRASS Development Team, and licensed under terms of the
GNU General Public License (GPL) version >=2.
 
This GRASS GIS 7.8.2 release is coordinated and produced by
the GRASS Development Team with contributions from all over the world.

This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU
General Public License for more details.



また、PythonからGRASS GISを使うために必要な`grass-session`というライブラリをインストールしてください。
```bash
pip install grass-session
```

Jupyter Notebookファイルと同じフォルダに`data/watershed/`フォルダを作成し、解析対象のDEMファイルを保存してください。

まずは必要なライブラリのインポートとパラメータを設定します。

In [2]:
from grass_session import Session
from grass.script import core as gcore
import os

data_dir = os.path.join('data', 'watershed')
raster_path = os.path.join(data_dir, 'dem_fuefuki.tif')
gisdb_path = os.path.join(data_dir, 'grass_db')
location = 'fuefuki'

# 通常GRASS GISでは、PERMANENTマップセットに一次データを保存し、
# ユーザー、テーマごとにマップセットを作成して作業を行いますが、
# ここではすべて同じPERMANENTマップセット上で作業を行います。
mapset = 'PERMANENT'

GRASSデータベースを作成します。データベースの作成には、扱うGISデータの座標系や空間的な範囲を設定する必要がありますが、これらはdem_fuefuki.tifファイルをもとに作成します。また、データベースの作成と同時にデータベースにdem_fuefuki.tifを登録します。

In [3]:
with Session(gisdb=gisdb_path,
             location=location,
             mapset=mapset,
             create_opts=raster_path):
    # import geotiff to PERMANENT mapset and change extent
    gcore.run_command('r.in.gdal',
                      flags='oe',
                      input=raster_path,
                      band=1,
                      output='dem',
                      overwrite=True)

データベースに`dem`という名前で`dem_fuefuki.tif`ファイルが登録されました。GRASSのデータベースの中身は[g.list](https://grass.osgeo.org/grass78/manuals/g.list.html)を用いた以下のコマンドで確認することができます。

In [4]:
with Session(gisdb=gisdb_path,
             location=location,
             mapset=mapset):
    print(gcore.read_command('g.list',
                             type='rast'))

dem



`type='rast'`とすることで、ラスタデータをリストアップすることができます。

次に、登録された`dem`というラスタデータに対して、GRASSの`r.watershed`モジュールを適用します。

In [5]:
# create a new location from raster file
with Session(gisdb=gisdb_path,
             location=location,
             mapset=mapset):
    # Run r.watershed command
    gcore.run_command('r.watershed',
                      elevation='dem',
                      threshold=1000,
                      accumulation='accumulation',
                      drainage='drainage',
                      basin='basin',
                      stream='stream',
                      half_basin='half_basin',
                      length_slope='length_slope',
                      slope_steepness='slope_steepness',
                      tci='tci',
                      spi='spi',
                      overwrite=True)
    # List up created rasters
    print(gcore.read_command('g.list',
                             type='rast'))

accumulation
basin
dem
drainage
half_basin
length_slope
slope_steepness
spi
stream
tci



`r.watarshed`コマンドのオプションは、今回QGISで実行したものと同じ値としました。各オプションの詳細は[GRASS GISの公式ドキュメント](https://grass.osgeo.org/grass78/manuals/r.watershed.html)を参照してください。

`r.watarshed`コマンドに続いて、`g.list`コマンドを実行し、作成したラスタデータの一覧を表示しています。

`grass.script`の`core.run_command()`と`core.read_command()`との違いは、単純にGRASSのモジュールを動かす場合は`run_command()`、GRASSモジュールの標準出力を文字列でとってきたいときは`read_command()`を利用します。詳細は[GRASS GISの公式wiki](https://grasswiki.osgeo.org/wiki/GRASS_Python_Scripting_Library#Uses_for_read.2C_feed_and_pipe.2C_start_and_exec_commands)に開設があります。

DEMから作成された河川のデータはstreamという名前でラスタデータとして保存されています。これを[r.to.vect](https://grass.osgeo.org/grass78/manuals/r.to.vect.html)モジュールを利用してベクトルデータに変換してみましょう。

In [6]:
with Session(gisdb=gisdb_path,
             location='fuefuki',
             mapset='PERMANENT'):
    gcore.run_command('r.to.vect',
                      input='stream',
                      output='stream',
                      type='line',
                      overwrite=True)

データベースの中身を確認します。

In [7]:
with Session(gisdb=gisdb_path,
             location='fuefuki',
             mapset='PERMANENT'):
    print(gcore.read_command('g.list',
                             type='vector'))

stream



続いて、データベース内に生成されたラスタデータをGeotiffファイルとして抽出します。GRASSのデータベースからラスタデータをファイルとして取り出すには、[r.out.gdalモジュール](https://grass.osgeo.org/grass78/manuals/r.out.gdal.html)を利用します。

In [8]:
with Session(gisdb=gisdb_path,
             location='fuefuki',
             mapset='PERMANENT'):
    gcore.run_command('r.out.gdal',
                      flags='mt',
                      input='accumulation',
                      output=os.path.join(data_dir, 'accumulation.tif'),
                      format='GTiff',
                      createopt='TFW=YES,COMPRESS=LZW',
                      overwrite=True)
    

`accumuration`の所を、`basin`や`drainage`に変更することで、そのほかの生成されたラスタデータをファイルとして取り出すことができます。

ベクタデータのファイルへの書き出しは[v.out.ogr](https://grass.osgeo.org/grass78/manuals/v.out.ogr.html)を利用します。

In [9]:
with Session(gisdb=gisdb_path,
             location='fuefuki',
             mapset='PERMANENT'):
    gcore.run_command('v.out.ogr',
                      flags='e',
                      input='stream',
                      output=os.path.join(data_dir, 'stream.shp'),
                      format='ESRI_Shapefile',
                      overwrite=True)

上のコードで`stream.shp`というShapefileが保存されます。

以上で、PythonからGRASS GISを使ってDEMデータの解析を行うことができました。